In [72]:
from pizzaro.gateway import send_command, wait_for_stepper_available, system_reset
import socket
import logging
import asyncio
import threading
import random
import time

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 9999))

In [68]:
async def pop_cheese(s):
    send_command(f'mmd move_to 200', s)
    wait_for_stepper_available(s)
    send_command(f'mmd belt_spd 200', s)
    send_command(f'mmd dispenser1_spd -300', s)
    spd = 600
    for i in range(1):
        send_command(f'mmd dispenser0_spd {spd}', s)
        await asyncio.sleep(10)
        send_command(f'mmd dispenser0_spd -{spd}', s)
        await asyncio.sleep(10)
    send_command(f'mmd dispenser0_off', s)
    send_command(f'mmd dispenser1_off', s)

await pop_cheese(s)

REQ:  b'{"command": "mmd move_to 200"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd belt_spd 200"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd dispenser1_spd -300"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd dispenser0_spd 600"}'
RESP:  b'{"Mmd":"MmdAck"}'


/tmp/ipykernel_1254113/2765449237.py:3: RuntimeWarning: coroutine 'wait_for_stepper_available' was never awaited
  wait_for_stepper_available(s)


REQ:  b'{"command": "mmd dispenser0_spd -600"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd dispenser0_off"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd dispenser1_off"}'
RESP:  b'{"Mmd":"MmdAck"}'


In [31]:
await system_reset(s)

REQ:  b'{"command": "mmd home"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd pr_off"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd pp_off"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd belt_off"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd dispenser0_off"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd dispenser1_off"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}

In [73]:
async def try_cheese():
    send_command(f'mmd home', s)
    send_command(f'mmd belt_off', s)
    send_command(f'mmd pr_off', s)
    await wait_for_stepper_available(s)

    def dispenser0_thread(s, speed, stop_dispenser0):
        while not stop_dispenser0.is_set():
            send_command(f'mmd dispenser0_spd {speed}', s)
            print(f"Current speed: {speed}")
            speed = -speed
            time.sleep(30)
        send_command(f'mmd dispenser0_off', s)

    # 创建停止事件
    stop_dispenser0 = threading.Event()
    
    # 创建并启动线程
    thread = threading.Thread(target=dispenser0_thread, args=(s, 600, stop_dispenser0,))
    thread.start()

    d1_spd = -300
    (onhold_secs, keep_secs) = (1, 4)
    print(d1_spd, onhold_secs, keep_secs)
    send_command(f'mmd dispenser1_spd {d1_spd}', s)
    for i in range(1):
        send_command(f'mmd move_to 100', s)
        send_command(f'mmd belt_spd 50', s)
        send_command(f'mmd pr_spd 70', s)
        await asyncio.sleep(onhold_secs)

        send_command(f'mmd move_to 130', s)
        send_command(f'mmd belt_spd 0', s)
        send_command(f'mmd pr_spd 100', s)
        await asyncio.sleep(0.2)
        send_command(f'mmd pr_spd 200', s)
        await asyncio.sleep(0.2)
        send_command(f'mmd pr_spd 300', s)
        await asyncio.sleep(0.2)
        send_command(f'mmd pr_spd 400', s)
        await asyncio.sleep(0.2)
        send_command(f'mmd pr_spd 500', s)
        await asyncio.sleep(0.2)
        send_command(f'mmd pr_spd 600', s)
        await asyncio.sleep(0.2)
        send_command(f'mmd pr_spd 700', s)
        send_command(f'mmd belt_spd 100', s)
        await asyncio.sleep(2)

    MIN_POS = 100
    PRODUCT = (540 - MIN_POS) * 50 / 75
    for p in range(140, 460, 60):
        # d1_spd = -1000 if d1_spd <= -1000 else d1_spd - 80
        send_command(f'mmd dispenser1_spd {d1_spd}', s)
        r = 540 - p
        a = (p - MIN_POS) * 2 + 50
        # a = 700
        v_b = round(a * r / PRODUCT) // 10 * 10
        print(d1_spd, p, a, v_b)
        send_command(f'mmd move_to {p}', s)
        send_command(f'mmd belt_spd 150', s)
        send_command(f'mmd pr_spd 700', s)
        await wait_for_stepper_available(s)
        await asyncio.sleep(keep_secs * r / 540)

    send_command(f'mmd move_to 460', s)
    stop_dispenser0.set()
    send_command(f'mmd dispenser0_off', s)
    send_command(f'mmd dispenser1_off', s)
    await asyncio.sleep(5)
    await wait_for_stepper_available(s)

    send_command(f'mmd home', s)
    send_command(f'mmd belt_off', s)
    send_command(f'mmd pr_off', s)
    await wait_for_stepper_available(s)

    thread.join()

await try_cheese()


REQ:  b'{"command": "mmd home"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd belt_off"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd pr_off"}'
RESP:  b'{"Mmd":"MmdAck"}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b'{"AtomiError":{"MmdUnavailable":"Unavailable"}}'
REQ:  b'{"command": "mmd wait_idle"}'
RESP:  b